In [1]:
import os
os.chdir('../cmuMine')
os.listdir()

['filter_patents.csv',
 '.DS_Store',
 'test',
 'indepn_depn',
 'val_sample.csv',
 'test_sample.csv',
 'filter_patents_.csv',
 'train_sample.csv',
 'val2006.csv',
 'train2006.csv',
 'test2006.csv',
 'val_sampl_balancede.csv',
 'test_sample_balanced.csv',
 'Helper doc.docx',
 'test_instruction_dataset.csv',
 'train',
 'train_sample_balanced.csv',
 'train_instruction_dataset.csv',
 'val_instruction_dataset.csv',
 'val']

In [2]:
import pandas as pd
train_dataset= pd.read_csv('train_sample_balanced.csv')
test_dataset= pd.read_csv('test_sample_balanced.csv')
val_dataset= pd.read_csv('val_sampl_balancede.csv')
print(train_dataset.shape)
print(test_dataset.shape)
print(val_dataset.shape)

(900, 6)
(135, 6)
(135, 6)


In [9]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import warnings
# Suppress warnings
warnings.filterwarnings('ignore')

# Load the test dataset
test_df = pd.read_csv('test_sample_balanced.csv')

# Initialize the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Ensure EOS token is used for padding if pad token is not defined
tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token

# Resize model embeddings in case pad token was added
model.resize_token_embeddings(len(tokenizer))

# Create a text generation pipeline
text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Function to generate text using the model for each abstract and summary
def generate_claim(abstract, summary, tokenizer, model, max_length):
    # Prepare the prompt with abstract and summary
    prompt = "Given the abstract and summary, generate the first independent claim:" \
             "\nAbstract: " + abstract + "\nSummary: " + summary
    
    # Tokenize and truncate the prompt to fit the model's input size
    inputs = tokenizer(prompt, return_tensors='pt', max_length=max_length, truncation=True)
    
    # Generate the output claim
    output = model.generate(**inputs, max_length=max_length, num_return_sequences=1)
    generated_claim = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Remove the original prompt from the output to get the generated claim only
    claim_start = generated_claim.find("Summary:") + len("Summary:")
    generated_claim = generated_claim[claim_start:].strip()
    return generated_claim

# Select the relevant portions of the text and generate the claim
test_df['generated_first_claim'] = test_df.apply(
    lambda x: generate_claim(x['abstract'], x['summary'], tokenizer, model, max_length=1024),
    axis=1
)

# Calculate the BLEU score
references = [[ref.split()] for ref in test_df['firstclaim']]
candidates = [cand.split() for cand in test_df['generated_first_claim']]
bleu_score = corpus_bleu(references, candidates, smoothing_function=SmoothingFunction().method1)

print(f"BLEU score: {bleu_score}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

BLEU score: 0.10177450989460743


In [10]:
test_df

,patent_number,abstract,summary,firstclaim,claims,cpc_class,generated_first_claim
0,US20060247247A1,The present invention provides pyrazole derive...,The present invention pertains to compounds h...,"1. A compound having the formula (I), \n\n\n\n...","1. A compound having the formula (I), \n\n\n\n...",A,The present invention pertains to compounds ha...
1,US20060266357A1,An oxygen concentrator for delivering consiste...,The present invention provides an apparatus t...,1. Apparatus comprising means for producing a ...,1. Apparatus comprising means for producing a ...,A,The present invention provides an apparatus th...
2,US20060204598A1,A method for producing a consumable nicotine-a...,The present invention resides in a method for...,1. A method for producing a consumable nicotin...,1. A method for producing a consumable nicotin...,A,The present invention resides in a method for ...
3,US20060261569A1,Mobility assist devices for moving an individu...,"Broadly, the present invention provides a plu...",1. A mobility assist device for moving an indi...,1. A mobility assist device for moving an indi...,A,"Broadly, the present invention provides a plur..."
4,US20060190104A1,"An electronic device which may used to enter, ...",An electronic device which may be used to ent...,1. An electronic mountable flip-down apparatus...,1. An electronic mountable flip-down apparatus...,A,An electronic device which may be used to ente...
...,...,...,...,...,...,...,...
130,US20060201003A1,A cutting implement having a pair of cutting b...,It is an object of the present invention to p...,1. A cutting implement comprising: \na pair of...,1. A cutting implement comprising: \na pair of...,Y,It is an object of the present invention to pr...
131,US20060201897A1,A shelving system includes an elongated mounti...,According to one aspect of the present invent...,1. A shelving system comprising: \nan elongate...,1. A shelving system comprising: \nan elongate...,Y,According to one aspect of the present inventi...
132,US20060260101A1,The invention relates to a positioning mechani...,The objective of the invention is to propose ...,1. Positioning mechanism for releasably fixing...,1. Positioning mechanism for releasably fixing...,Y,The objective of the invention is to propose a...
133,US20060251887A1,An antifriction composite system (1) for a bea...,The present invention relates to an antifrict...,1. An antifriction composite multilayer system...,1. An antifriction composite multilayer system...,Y,The present invention relates to an antifricti...


In [15]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset, load_metric

# Initialize the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Ensure EOS token is used for padding if pad token is not defined
tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token

# Resize model embeddings in case pad token was added
model.resize_token_embeddings(len(tokenizer))


# Function to tokenize the input and label pairs for training
def tokenize_pairs(examples):
    # Tokenization will automatically add the special tokens for GPT-2
    model_inputs = tokenizer(examples['input_text'], max_length=1024, truncation=True, padding='max_length')
    model_inputs['labels'] = tokenizer(examples['labels'], max_length=1024, truncation=True, padding='max_length').input_ids
    return model_inputs

# Prepare the datasets
def prepare_dataset(csv_file):
    df = pd.read_csv(csv_file)
    df['input_text'] = "Abstract: " + df['abstract'] + " [SEP] Summary: " + df['summary']  # Create a prompt
    df['labels'] = df['firstclaim'].tolist()  # Expected output
    return df

# Load and prepare datasets
train_df = prepare_dataset('train_sample_balanced.csv')
val_df = prepare_dataset('val_sampl_balancede.csv')
test_df = prepare_dataset('train_sample_balanced.csv')

# Tokenize datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(tokenize_pairs, batched=True)
val_dataset = val_dataset.map(tokenize_pairs, batched=True)
test_dataset = test_dataset.map(tokenize_pairs, batched=True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

In [16]:
train_dataset

Dataset({
    features: ['patent_number', 'abstract', 'summary', 'firstclaim', 'claims', 'cpc_class', 'input_text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 900
})

In [18]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    early_stopping_patience=2
)

# Initialize the model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained('./finetuned_gpt2_patent')
tokenizer.save_pretrained('./finetuned_gpt2_patent')


  0%|          | 0/1350 [00:00<?, ?it/s]

{'loss': 10.6079, 'learning_rate': 1e-05, 'epoch': 0.02}
{'loss': 4.7583, 'learning_rate': 2e-05, 'epoch': 0.04}
{'loss': 1.3094, 'learning_rate': 3e-05, 'epoch': 0.07}
{'loss': 1.0335, 'learning_rate': 4e-05, 'epoch': 0.09}
{'loss': 1.0382, 'learning_rate': 5e-05, 'epoch': 0.11}
{'loss': 1.0042, 'learning_rate': 4.961538461538462e-05, 'epoch': 0.13}
{'loss': 1.4264, 'learning_rate': 4.923076923076924e-05, 'epoch': 0.16}
{'loss': 1.3512, 'learning_rate': 4.884615384615385e-05, 'epoch': 0.18}
{'loss': 1.1543, 'learning_rate': 4.846153846153846e-05, 'epoch': 0.2}
{'loss': 1.0932, 'learning_rate': 4.8076923076923084e-05, 'epoch': 0.22}
{'loss': 1.0358, 'learning_rate': 4.76923076923077e-05, 'epoch': 0.24}
{'loss': 0.849, 'learning_rate': 4.730769230769231e-05, 'epoch': 0.27}
{'loss': 1.2313, 'learning_rate': 4.692307692307693e-05, 'epoch': 0.29}
{'loss': 0.8991, 'learning_rate': 4.653846153846154e-05, 'epoch': 0.31}
{'loss': 0.825, 'learning_rate': 4.615384615384616e-05, 'epoch': 0.33}
{'

  0%|          | 0/34 [00:00<?, ?it/s]

{'eval_loss': 0.867717444896698, 'eval_runtime': 65.4196, 'eval_samples_per_second': 2.064, 'eval_steps_per_second': 0.52, 'epoch': 1.0}
{'loss': 0.8974, 'learning_rate': 3.4230769230769234e-05, 'epoch': 1.02}
{'loss': 0.7272, 'learning_rate': 3.384615384615385e-05, 'epoch': 1.04}
{'loss': 0.919, 'learning_rate': 3.346153846153846e-05, 'epoch': 1.07}
{'loss': 0.9717, 'learning_rate': 3.307692307692308e-05, 'epoch': 1.09}
{'loss': 1.2155, 'learning_rate': 3.269230769230769e-05, 'epoch': 1.11}
{'loss': 0.8276, 'learning_rate': 3.230769230769231e-05, 'epoch': 1.13}
{'loss': 0.8378, 'learning_rate': 3.192307692307692e-05, 'epoch': 1.16}
{'loss': 1.0223, 'learning_rate': 3.153846153846154e-05, 'epoch': 1.18}
{'loss': 0.8595, 'learning_rate': 3.115384615384615e-05, 'epoch': 1.2}
{'loss': 0.743, 'learning_rate': 3.0769230769230774e-05, 'epoch': 1.22}
{'loss': 1.0427, 'learning_rate': 3.0384615384615382e-05, 'epoch': 1.24}
{'loss': 1.0077, 'learning_rate': 3e-05, 'epoch': 1.27}
{'loss': 0.9342

  0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/34 [00:00<?, ?it/s]

{'eval_loss': 0.8375254273414612, 'eval_runtime': 292.9751, 'eval_samples_per_second': 0.461, 'eval_steps_per_second': 0.116, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


{'train_runtime': 84443.2577, 'train_samples_per_second': 0.032, 'train_steps_per_second': 0.016, 'train_loss': 1.0744552357991537, 'epoch': 3.0}


KeyError: 0

In [19]:
os.getcwd()

'/Users/sachin.murthy/Desktop/cmuMine'

In [20]:
os.listdir()

['filter_patents.csv',
 '.DS_Store',
 'test',
 'indepn_depn',
 'val_sample.csv',
 'test_sample.csv',
 'filter_patents_.csv',
 'train_sample.csv',
 'val2006.csv',
 'train2006.csv',
 'test2006.csv',
 'results',
 'val_sampl_balancede.csv',
 'test_sample_balanced.csv',
 'logs',
 'Helper doc.docx',
 'test_instruction_dataset.csv',
 'finetuned_gpt2_patent',
 'train',
 'train_sample_balanced.csv',
 'train_instruction_dataset.csv',
 'val_instruction_dataset.csv',
 'val']

In [49]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline
import pandas as pd
from datasets import load_metric

# Load the fine-tuned model and tokenizer
model_path = './finetuned_gpt2_patent'
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Load the test dataset
test_df= pd.read_csv('test_sample_balanced.csv')  # Replace with the path to your actual test dataset

# Ensure EOS token is used for padding if pad token is not defined
tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token

# Resize model embeddings in case pad token was added
model.resize_token_embeddings(len(tokenizer))

# Create a text generation pipeline
text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Function to generate text using the model for each abstract and summary
def generate_claim(abstract, summary, tokenizer, model, max_length):
    # Prepare the prompt with abstract and summary
    prompt = "Given the abstract and summary, generate the firstclaim:" \
             "\nAbstract: " + abstract + "\nSummary: " + summary
    
    # Tokenize and truncate the prompt to fit the model's input size
    inputs = tokenizer(prompt, return_tensors='pt', max_length=max_length, truncation=True)
    
    # Generate the output claim
    output = model.generate(**inputs, max_length=max_length, num_return_sequences=1)
    generated_claim = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Remove the original prompt from the output to get the generated claim only
    claim_start = generated_claim.find("Summary:") + len("Summary:")
    generated_claim = generated_claim[claim_start:].strip()
    return generated_claim

# Select the relevant portions of the text and generate the claim
test_df['generated_first_claim'] = test_df.apply(
    lambda x: generate_claim(x['abstract'], x['summary'], tokenizer, model, max_length=1024),
    axis=1
)

# Calculate the BLEU score
references = [[ref.split()] for ref in test_df['firstclaim']]
candidates = [cand.split() for cand in test_df['generated_first_claim']]
bleu_score = corpus_bleu(references, candidates, smoothing_function=SmoothingFunction().method1)

print(f"BLEU score: {bleu_score}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

BLEU score: 0.8454613277509645
